# Toy Cipher를 이용한 차분 분석 예제



## Toy Cipher

예제에서 사용할 Toy Cipher는 다음 링크에서 설명한 자료를 토대로 만든 것입니다.

http://www.secmem.org/blog/2019/04/08/차분-공격의-이해/

이 Toy Cipher는 12비트 블록암호로, 3라운드로 구성되어 있으며, 

12비트씩 4개의 라운드키를 필요로합니다.

라운드키 확장 함수는 따로 없습니다.

ToyCipher의 구조를 그림으로 표현하면 아래와 같습니다.

![Toy Cipher 구조](images/ToyCipher.png)

여기에서 사용한 4-bit S-Box는 다음과 같습니다.

 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | a | b | c | d | e | f
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---
 6 | 7 | b | c | 9 | 8 | 4 | 0 | e | 5 | 3 | d | 1 | 2 | f | a

비트 치환은 다음과 같이 이루어집니다.

 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 
---|---|---|---|---|---|---|---|---|---|---|---
 0 | 2 | 4 | 6 | 8 | 10 | 1 | 3 | 5 | 7 | 9 | 11

In [1]:
# 필요한 패키지를 import 합니다.
import differential_cryptanalysis as dc
import toycipher as tc

In [2]:
# Toy Cipher의 객체를 하나 생성하고, 48비트 키를 랜덤하게 채워줍니다.
cipher = tc.ToyCipher()
cipher.random_keys()

In [3]:
# 차분 분석을 수행하기 위해 S-Box의 입-출력 테이블을 출력합니다.
dc.print_inout_table(cipher.SBOX)

SBox:  [6, 7, 11, 12, 9, 8, 4, 0, 14, 5, 3, 13, 1, 2, 15, 10]

입력쌍-출력차분 테이블
     0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15
0    0   1  13  10  15  14   2   6   8   3   5  11   7   4   9  12
1    1   0  12  11  14  15   3   7   9   2   4  10   6   5   8  13
2   13  12   0   7   2   3  15  11   5  14   8   6  10   9   4   1
3   10  11   7   0   5   4   8  12   2   9  15   1  13  14   3   6
4   15  14   2   5   0   1  13   9   7  12  10   4   8  11   6   3
5   14  15   3   4   1   0  12   8   6  13  11   5   9  10   7   2
6    2   3  15   8  13  12   0   4  10   1   7   9   5   6  11  14
7    6   7  11  12   9   8   4   0  14   5   3  13   1   2  15  10
8    8   9   5   2   7   6  10  14   0  11  13   3  15  12   1   4
9    3   2  14   9  12  13   1   5  11   0   6   8   4   7  10  15
10   5   4   8  15  10  11   7   3  13   6   0  14   2   1  12   9
11  11  10   6   1   4   5   9  13   3   8  14   0  12  15   2   7
12   7   6  10  13   8   9   5   1  15   4   2  12   

In [4]:
# 위에서 생성한 입-출력 테이블을 바탕으로 입력차분-출력차분 확률 분포표를 출력합니다.
dc.print_differential_prob_table(cipher.SBOX)

SBox:  [6, 7, 11, 12, 9, 8, 4, 0, 14, 5, 3, 13, 1, 2, 15, 10]

입력차분-출력차분 빈도표
     0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15
0   16  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0
1    0  4  0  2  2  2  0  2  0  0   0   2   0   0   2   0
2    0  0  0  0  0  0  0  0  6  0   0   2   0   6   2   0
3    0  0  0  2  0  0  2  0  0  2   2   2   4   2   0   0
4    0  0  0  0  0  0  0  4  0  0   0   0   4   0   0   8
5    0  0  2  0  2  0  0  0  2  2   0   2   2   0   4   0
6    0  2  6  0  2  0  0  2  0  0   0   0   0   0   0   4
7    0  2  0  4  2  2  2  0  0  0   2   0   2   0   0   0
8    0  2  2  0  0  0  0  0  6  0   4   2   0   0   0   0
9    0  0  0  2  0  0  2  0  0  4   0   2   0   0   2   4
10   0  0  4  0  0  6  2  0  0  2   2   0   0   0   0   0
11   0  2  2  2  2  0  2  2  0  0   0   2   0   0   2   0
12   0  0  0  0  2  2  2  6  0  0   0   0   0   4   0   0
13   0  2  0  4  2  0  4  0  0  2   0   0   2   0   0   0
14   0  0  0  0  0  4  0  0  0  2   6   0   0   2   2

## 차분 경로 찾기 1

입력차분-출력차분 빈도표를 이용해서 발생할 확률이 높은 차분 경로를 탐색합니다.

![첫 번째 S-Box로 오는 차분 경로](images/Trail1.png)

위 그림은 (0x2, 0x0, 0x0)의 입력차분이 (0x8, 0x0, 0x0)의 출력차분으로 올 경로를 의미하며, 

이 때의 확률은 6/16 * 6/16 = 9/64 입니다.

이는 (0x2, 0x0, 0x0)의 입력 차분을 가지는 임의의 64개의 평문쌍 중 9개는 해당 차분 경로를 타고 계산될 것이라고 기대할 수 있습니다.

이 경로를 타게 되면 두번째와 세번째의 nibble의 출력 차분은 0이 되어야 하기 때문에 해당 부분의 차분이 0이 아닌 평문쌍은 버리고,

첫 번째 nibble에 해당하는 라운드 키를 추측하여, 한 라운드 복호화를 수행한 결과의 차분이 0x8이 되도록 하는 경우의 개수를 셉니다.

가장 많읜 평문쌍의 한라운드 복호화 결과의 차분이 0x8이 되도록 하는 키가 높은 확률로 해당 부분의 라운드 키가 됩니다.

In [5]:
# 첫 번째 nibble에 해당하는 라운드키 찾기
dc.try_recover_key(cipher, 32, input_diff=(2, 0, 0), target_diff=(8, 0, 0), key_mask=(0xf, 0, 0))

Key Count  [0x0 0x0 0x0 0x0 0x2 0x0 0x2 0x0 0x0 0x0 0x0 0x0 0x4 0x1 0x3 0x0]
Partial Key Candidates --> 1) 0xc 2) 0xe



![두 번째 nibble에 해당하는 차분 경로](images/Trail2.png)

In [6]:
# 두 번째 nibble에 해당하는 라운드키 찾기
dc.try_recover_key(cipher, 32, input_diff=(0, 2, 0), target_diff=(0, 0x4, 0), key_mask=(0, 0xf, 0))

Key Count  [0x2 0x6 0x3 0x3 0x1 0x0 0x3 0x0 0x0 0x3 0x0 0x1 0x3 0x3 0x6 0x2]
Partial Key Candidates --> 1) 0x1 2) 0xe



![세 번째 nibble에 해당하는 차분 경로](images/Trail13.png)

In [7]:
# 세 번째 nibble에 해당하는 라운드키 찾기
dc.try_recover_key(cipher, 128, input_diff=(0, 0, 2), target_diff=(5, 0, 0xa), key_mask=(0, 0, 0xf))

Key Count  [0x0 0x0 0x0 0x3 0x0 0x1 0x0 0x0 0x0 0x2 0x1 0x0 0x0 0x0 0x0 0x1]
Partial Key Candidates --> 1) 0x3 2) 0x9



In [8]:
# 실제 라운드 키
print(cipher.rks)

[[0x4 0x3 0x4]
 [0xb 0x1 0x6]
 [0x1 0x2 0x7]
 [0xc 0x1 0x3]]
